# Library import 

In [ ]:
import os
import random
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# pytroch
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler


# utils
from tqdm.auto import tqdm
import warnings
import wandb
from datetime import datetime
import re
from typing import Tuple

warnings.filterwarnings("ignore")

In [137]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [138]:
train = pd.read_csv("./data/train.csv")
train = train.iloc[:100]
train.drop(["ID", "제품"], axis=1, inplace=True)

In [139]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train[numeric_cols].min(axis=1)
max_values = train[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train[numeric_cols] = (train[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [140]:
encoder = LabelEncoder()
categorical_col = ["대분류", "중분류", "소분류", "브랜드"]

for col in categorical_col:
    train[col] = encoder.fit_transform(train[col])

In [141]:
def make_train_data(data, train_size=90, predict_size=21):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
    return input_data, target_data

In [142]:
def make_predict_data(data, train_size=21):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [143]:
train_input, train_target = make_train_data(train)
test_input = make_predict_data(train)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [144]:
train_target.shape

(35300, 21)

In [145]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [146]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((28240, 90, 5), (28240, 21), (7060, 90, 5), (7060, 21), (100, 21, 5))

In [147]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [148]:
train_dataset = CustomDataset(train_input, train_target)
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_dataloader = DataLoader(val_dataset, batch_size = 32, shuffle=False, num_workers=0)

In [149]:
for sample in train_dataloader:
    print(sample[0].shape)
    print(sample[1].shape)
    break

torch.Size([32, 90, 5])
torch.Size([32, 21])


# Define Model

###  Mulitple Timeseries Forecasting 

-> 날씨 : 90일치가지고 하루 </br>
-> 90일치로 21일을 예측

- 구조
    - 인코더
    - 어텐션
    - 디코더
    - 시퀀스 투 시퀀스 with 어텐션

- X : Batch_size, src_len, input_size
- y : Batch_size, trg_len

- 기계번역 소스 (원문)
- src : source
- trg : target

In [152]:
import torch.nn as nn
import torch

class Encoder(nn.Module):
    def __init__(self, input_dim: int, enc_hid_dim: int, dec_hid_dim:int, n_layers: int, dropout: float):
        '''
        Seq2Seq with Attention -> 이게 정확한 모델 명칭
        기존의 Seq2Seq 구조에서 어텐션을 결합하여 시계열 문제에서 발생할 수 있던 Long Term Dependecy 문제 해결시도
        따라서 encoder에서 hidden state를 디코더로 전달, 디코더에서 인코더와 attention연산을 통하여 attention weight를 확보한다.
        deocder에서 attention weight와 hidden state를 결합하여 모든 시퀀스에 대하여 집중을 할 수 있어졌음
            -> 멀어질 수록 약해지던 Seq2Seq 문제 해결

        parameter
            - input_dim : input 차원
            - enc_hid_dim : 인코더의 hidden_state 차원 -> 하이퍼파라미터 변경 필요
            - dec_hid_dim : 디코더의 hidden_state 차원 -> 하이퍼파라미터 변경 필요
               **주의 사항**
                - 만약 enc_hid_dim과 dec_hid_dim이 일치하지 않을 때 에러 발생할 수 도 있음 -> 실험은 안해봤는데 그럴 가능성 존재 -> assert로 처리 혹은 그냥 모델 구성시 일치시키는 방안 추천
            - n_layer : lstm 층을 몇개를 쌓을지
            - dropout : dropout 확률
        '''
        super().__init__()

        # 데이터 셋의 경우 [batch_size, window_size, input_size]로 구성되어 있음, 따라서 batch_first옵션을 True로 해야 에러가 발생안함
        # 인코더 층에는 bidirectional 옵션을 걸었는데 이는 한방향 학습이 아닌 양방향 학습을 통해 더 정확한 정보를 전달하기 위함
        # 만일 이 옵션을 False로 할경우 아래 Linear 층 곱하기를 제외하면 에러 해결 될듯? -> 확실치 않기 때문에 디버깅 필요
        self.rnn = nn.LSTM(input_dim, enc_hid_dim, n_layers, bidirectional=True, batch_first=True)
        self.fc_hidden = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.fc_cell = nn.Linear(enc_hid_dim * 2, dec_hid_dim)  
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # LSTM 의 경우 ouput과 hidden state, cell state 총 3가지를 반환
        # 일반 LSTM 모델의 경우 output을 제외한 나머지 두개는 필요없기 때문에 output, _ = lstm(x)로 보통 사용하지만 이 경우 디코더에 시계열 정보를 넘겨줘야 하기 때문에 추가 연산을 통하여 넘겨줌
        outputs, (hidden, cell) = self.rnn(src)
        

        # 카톡으로 보낸 사진 구현
        hidden = torch.tanh(self.fc_hidden(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)))
        cell = torch.tanh(self.fc_cell(torch.cat((cell[-2, :, :], cell[-1, :, :]), dim=1)))

        return outputs, (hidden, cell)


class Attention(nn.Module):
    
    def __init__(self, enc_hid_dim: int, dec_hid_dim: int):

        '''
        다양한 어텐션이 존재, dot product 어텐션 부터, global 어텐션 등등.. 하지만 실력 부족으로 인한 concat 어텐션 밖에 구현... -> 금요일 전까지 다른 어텐션 코드 찾아볼 예정
        enc_hid_dim : 인코더 히든 차원
        dec_hid_dim : 디코더 히든 차원
        어텐션의 경우 어텐션과 함께 value를 반환하여 이 두개를 통해 어텐션 가중치를 얻어야함
        '''
        super().__init__()
        # 공식대로 구현
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # encoder_outputs = [batch_size, src_len, input_size]
        # 따라서 배치 사이즈와 타겟 길이를 추출
        batch_size = encoder_outputs.size(0)
        src_len = encoder_outputs.size(1)
    
        # 디버깅 상당히 오래한 부분
        # squeeze를 통하여 먼저 첫번째 차원을 제거하고 두번째 차원을 확장하면서 행렬 합을 할 수 있게 차원을 맞추는 부분
        # 물론 대체 왜 아까 (1, 2880, 1) 인지는 이해하지 못하였지만 아무튼 해결
        # print(f"시작 hidden 의 크기 :{hidden.shape}")
        hidden = hidden.squeeze(0)
        # print(f"첫번째 차원을 없앤 hidden 의 크기 : {hidden.shape}")
        hidden = hidden.unsqueeze(1).expand(-1, src_len, -1)
        # print(f"차원을 맞춘 hidden 의 크기 : {hidden.shape}")
    
        # 보통의 코드에서 energy라고 표현해서 변수명을 에너지로 사용
        # 어텐션 가중치를 얻기 위하여 인코더의 출력과 인코더의 히든 스테이트를 행렬합을 하고 어텐션 계산을 통해 어텐션 벡터 구하는 과정
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        # 차원을 맞춰 최종 어텐션 스코어를 확보한다.
        attention = self.v(energy).squeeze(2)
        # 소프트 맥스를 적용하여 어텐션 스코어에서 어텐션 가중치를 확보한다. 
        return nn.functional.softmax(attention, dim=1)

class Decoder(nn.Module):
    def __init__(self, output_dim: int, enc_hid_dim: int, dec_hid_dim: int, n_layers: int, dropout: float, attention: nn.Module):
        '''
        디코더의 경우 시퀀스투시퀀스 구조와 비슷하게 인코더에서 hidden_state를 입력받아 그것을 통하여 디코더에 생성 (주로 기계번역) 하지만 시퀀스가 길어지면 처음 넣었던 정보가 잘 담기지 못하는 long term dependency
        문제 발생 따라서 디코더의 모든 차원에 어텐션 가중치를 더해 출력의 정확도를 높임
        디코더 이기 때문에 마지막 출력 부분은 Linear 함수를 통하여 prediction값을 생성하도록 코딩하였음
        
        parameter 
            - output_dim : output_dim 출력 차원 21고정 상수값
            - enc_hid_dim : 인코더 히든 차원 -> 하이퍼파라미터
            - dec_hid_dim : 디코더 히든 차원 -> 하이퍼파라미터 -> 하지만 위에서 언급한것처럼 서로 다를때 실험안해봤기 때문에 서로 다를경우 제보 바랍니다.
            - n_layer : 디코더 레이어 개수 -> 하이퍼파라미터 -> 쓰다보니 이것도 인코더랑 다르게 생성할 생각을 못했지만 혹시나 서로 다르면 에러 발생할 수 있기에 에러나면 제보 바랍니다.
            - dropout : dropout prob
            - attetntion : 어텐션 모듈을 불러와서 어텐션 가중치를 계산해야함
        '''
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        # target 값도 [batch_size, window_size]로 구성되어 있기 때문에 batch_first옵션을 사용
        # 그래서 학습시 타겟의 차원을 늘려 [batch_size, window_size, 1]로 만들어줘야함 -> 학습코드에 추가했습니다
        # 여기에서는 bidirectional 옵션을 사용하지 않고 default 값인 false로 하였는데 이걸 양방향으로 예측하는 경우는 아마 보지 못했는데 에러나면 제보 바랍니다.
        self.rnn = nn.LSTM((enc_hid_dim * 2) + output_dim, dec_hid_dim, n_layers, batch_first=True)
        # prediction을 위한 Linear 층
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + output_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell, encoder_outputs, hidden_last_layer):
        # 차원 맞추기
        input = input.unsqueeze(1)
        
        # 실제로 인코더의 모든 hidden_state가 아닌 마지막 만을 받아와서 어텐션 가중치를 구하기 때문에 추가 변수 지정
        attention_weights = self.attention(hidden_last_layer, encoder_outputs)
        # 차원 맞추기
        attention_weights = attention_weights.unsqueeze(1)
        # bmm -> matmul과 매우 유사 하지만 bmm은 3차원 텐서에 관해서만 행렬 합을 할 수 있으며 브로드캐스팅을 지원안하는 것으로 알고있습니다. 아마..?
        weighted = torch.bmm(attention_weights, encoder_outputs)
        # rnn_input으로 들어갈 입력데이터와 가중치를 행렬합을 함
        # 이렇게 해야 디코더의 모든 시퀀스에 대하여 어텐션 가중치를 적용가능
        rnn_input = torch.cat((input, weighted), dim=2)
        output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # 차원 맞추기
        embedded = input.squeeze(1)
        output = output.squeeze(1)
        weighted = weighted.squeeze(1)
        # 임베딩과, 결과물과, 가중치를 행렬합 계산을 하고 Linear층을 통하여 예측함
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))

        return prediction, (hidden, cell)

class Seq2Seq(nn.Module):
    def __init__(self, encoder: nn.Module, decoder: nn.Module, device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.relu = nn.ReLU()

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # shape 지정
        trg_len = trg.shape[1]
        batch_size = trg.shape[0]
        trg_dim = trg.shape[2]
        # 0행렬을 만들어서 최종 결과물 shape의 형태의 텐서 생성
        outputs = torch.zeros(batch_size, trg_len, trg_dim).to(self.device)
        # src -> 입력데이터, source 즉 90일치 (예시, 윈도우 사이즈, 변경가능)의 데이터를 통하여 인코더의 결과물과 히든 스테이트, 셀스테이트를 리턴
        encoder_outputs, (hidden, cell) = self.encoder(src)


        # 타겟데이터에서 입력으로 넣을 것만 추출
        # trg = [batch_size, trg_len, 1]
        input = trg[:, 0, :]
        # print(f"input 의 차원 : {input.shape}")
        
        # 아래 반복을 한 이유
        # 처음 모델을 제작할 시 n_layer 옵션을 걸지 않고 단순한 한층 구조를 만들어서 (n_layer=1) 하다보니 차원이 무조건 [1, batch_size, len]으로 고정
        # 하지만 모델 고도화 작업 중 단층은 너무 별로지 않나 라는 생각을 하였고 하다보니 [n_layer, batch_size, len] 구조를 만들기 위한 reepat문으로 차원수를 맞춰주었음
        hidden = hidden.unsqueeze(0).repeat(self.decoder.rnn.num_layers, 1, 1)
        cell = cell.unsqueeze(0).repeat(self.decoder.rnn.num_layers, 1, 1)
        # 층이 여러개다 보니까 마지막 층의 hidden_state 만사용하기 위하여 따로 변수로 추출
        hidden_last_layer = hidden[-1]

        for t in range(1, trg_len):
            
            # 0행렬을 만들고 trg_len만큼의 길이만큼 반복하면서 각 time step별로 0행렬을 채워넣는 구조
            # decoder forward 부분에 필요한 파라미터를 입력 인자로 결과물 추출
            output, (hidden, cell) = self.decoder(input, hidden, cell, encoder_outputs, hidden_last_layer)
            # 각 time step 별 결과물을 채움
            outputs[:, t, :] = output
            
            # 교사 강요 (teacher forcing) 이러한 인코더 디코더 구조의 경우 너무 학습 속도가 느리다는 단점이 존재한다.
            # 이를 해결하기 위해서 교사 강요를 사용해 학습 속도를 높였음
            # 하지만 불안정한 학습으로 수렴할 수 있기 때문에 제거 고려해야함
            # 원래라면 이러한 시퀀스 모델은 자기회귀 모델이기 때문에 첫번째 결과물이 두번째 입력값으로 들어가야한다.
            # 이럴경우 21일을 예측하려면 21번의 시퀀스를 그대로 진행해야함
            # 따라서 속도를 높이기 위하여 입력값의 경우 teacher_forcing_ratio 만큼 고정하여 학습을 하여 속도를 올린다. -> 한국말이지만 제가 읽어도 모르겠기 때문에 이해안가면 말해주세요
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            input = trg[:, t, :] if teacher_force else output

        return outputs

In [153]:
INPUT_DIM = 5
OUTPUT_DIM = 1
N_LAYER = 4
ENC_HID_DIM = 32
DEC_HID_DIM = 32
DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYER, DROPOUT)
dec = Decoder(OUTPUT_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYER, DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [154]:
# Warmup Scheduler
class WarmupLR(optim.lr_scheduler.LambdaLR):

    def __init__(
        self,
        optimizer: optim.Optimizer,
        warmup_end_steps: int,
        last_epoch: int = -1,
    ):
        
        def wramup_fn(step: int):
            if step < warmup_end_steps:
                return float(step) / float(max(warmup_end_steps, 1))
            return 1.0
        
        super().__init__(optimizer, wramup_fn, last_epoch)


In [155]:
model_name = type(model).__name__

# define loss
loss_function = nn.MSELoss()

# define optimizer
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer_name = type(optimizer).__name__

# define scheduler
# scheduler = WarmupLR(optimizer, 1500)
scheduler = None
scheduler_name = type(scheduler).__name__ if scheduler is not None else "no"

max_epoch = 2

In [156]:
clip_value = 1.0

In [157]:
def train(model, optimizer, train_dataloader, val_dataloader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    
    for epoch in range(1, 2):
        model.train()
        train_loss = []
        
        for X, Y in tqdm(iter(train_dataloader)):
            X = X.to(device)
            Y = Y.to(device)
            # y의 경우 현재 [batch_size, 21]로 구성 unsqueeze를 통해 3번째 차원 (인덱스 2)를 만들어줘야함
            Y = Y.unsqueeze(2)
            # 이렇게 하면 [batch_size, 21, 1] 로 변시ㅣㄴ
            
            # Foward
            optimizer.zero_grad()
            # get prediction
            # **주의사항**
            # 모델 학습시 x, y 둘다 넣어 줘야합니다.
            output = model(X, Y)
            
            loss = criterion(output, Y)
            
            # back propagation
            loss.backward()
            # Apply gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            # Perform LR scheduler Work
            if scheduler is not None:
                scheduler.step()
            
            train_loss.append(loss.item())
        
        val_loss = validation(model, val_dataloader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
        
        # # WandB logging
        # wandb.log({
        #     "Epoch": epoch,
        #     "Train Loss": np.mean(train_loss),
        #     "Validation Loss": val_loss,
        # })
        
    return best_model

def validation(model, val_dataloader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_dataloader)):
            X = X.to(device)
            Y = Y.to(device)
            Y = Y.unsqueeze(2)

            output = model(X, Y)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [158]:
infer_model = train(model, optimizer, train_dataloader, val_dataloader, device)

  0%|          | 0/883 [00:00<?, ?it/s]

  0%|          | 0/221 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.01401] Val Loss : [0.01342]
Model Saved


In [159]:
test_dataset = CustomDataset(test_input, None)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle=False, num_workers=0)

- 추론이 조금 많이 변경됨
- 기존의 경우 모델에 X값만 넣으면 되는데 이 모델의 경우 src와 trg두개를 넣어야 학습이 된다.
- 따라서 기존의 model(x) 만 입력하면 되는 추론에서 직접 마지막 deocder의 forward 구조를 조금 변경하여서 추론 코드를 작성하였음

In [160]:
def inference(model, src, max_len=21):
    model.eval() 
    src = src.to(device)
    src_len = src.shape[1]

    
    encoder_outputs, hidden_cell = model.encoder(src)

    hidden = hidden_cell[0].unsqueeze(0).repeat(N_LAYER, 1, 1)
    cell = hidden_cell[1].unsqueeze(0).repeat(N_LAYER, 1, 1)
    hidden_last_layer = hidden[-1]
    input = src[:, 0, :]

    outputs = []
    
    for t in range(max_len):
        
        input_dim = input.shape[-1]
        if input_dim != OUTPUT_DIM:
            
            input = input[:, :OUTPUT_DIM]
        output, (hidden, cell) = model.decoder(input, hidden, cell, encoder_outputs, hidden_last_layer)
        outputs.append(output.unsqueeze(1))
        
        input = output

    return torch.cat(outputs, dim=1)

In [161]:
def evaluate(model, test_dataloader):
    model.eval() 

    all_predictions = []

    with torch.no_grad():  
        for src in test_dataloader:
            src = src.to(device)
            predictions = inference(model, src)
            all_predictions.append(predictions)

    return torch.cat(all_predictions, dim=0)

all_predictions = evaluate(infer_model, test_dataloader)

print(all_predictions.shape) 

torch.Size([100, 21, 1])


In [162]:
pred = all_predictions.squeeze(2).cpu().numpy()

In [163]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

# 학습하다보니 0이하인 값이 많이 발생
# -> 물론 에포크 한번만 돌리고 데이터도 100개로 했음
# 아무튼 판매량인데 0이하가 말이되나 하고 후처리 하나 추가
pred[pred < 0] = 0

In [164]:
pred

array([[ 8,  6,  3, ...,  2,  2,  2],
       [ 9,  6,  3, ...,  2,  2,  2],
       [40, 24, 14, ...,  7,  7,  7],
       ...,
       [ 8,  5,  3, ...,  1,  1,  1],
       [ 2,  2,  1, ...,  4,  4,  4],
       [ 0,  0,  0, ...,  0,  0,  0]])